In [ ]:
import os
import osmnx as ox
import geopandas as gpd
import pandas as pd
import shapely
from shapely.geometry import *
import matplotlib.pyplot as plt
import networkx as nx
import math
import PIL
from PIL import Image
import cv2
import numpy as np
from ultralytics import YOLO

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# os.chdir(r"C:\Users\danil\PycharmProjects\pythonProject3")
pd.set_option("display.max_columns", 99)
pd.set_option('display.max_colwidth', None)

In [ ]:
# Импорт файла с ПЗЗ
zon = gpd.read_file(r"C:...")  
zon = zon.to_crs(crs = 4326)
zon

In [ ]:
# Берём координатную точку, в радиусе 200 метров беру данные про здания. 
def context_data(text_file_path):
    # Создание пустого геодатафрейма
    context = gpd.GeoDataFrame(columns=['Имя изображения', 'geometry', 'Код зоны по ПЗЗ', 'Тип зоны', 'Преобладающий тип застройки', 'Тип ближайшей дороги'], crs="EPSG:4326")

    # Инициализация переменных H и b
    H = None
    b = None

    # Чтение данных из файла
    with open(text_file_path, 'r') as file:
        for line in file:
            # Обработка строки с параметрами H и b
            if line.startswith('H'):
                H = float(line.strip().split()[1])
            elif line.startswith('b'):
                b = float(line.strip().split()[1])
            else:
                # Извлечение данных из строки
                image_name, coordinates = line.strip().split(' ')
                X_base, Y_base = map(float, coordinates.split(','))
                x = [X_base, Y_base]    #[60.013649, 30.373220]
                # зона по ПЗЗ 
                intersects = zon['geometry'].intersects(Point(x[1], x[0]))
                intersecting_zones = zon[intersects].values[0]
                # преобладающий тип застройки
                tags = {'building': True}
                distance = 100
                point = (x[0], x[1])
                gdf = ox.geometries_from_point(point, tags = {'building': True}, dist = distance).reset_index()
                gdf = gdf[gdf['element_type']=='way']
                gdf = gdf[gdf['geometry'].apply(lambda x : x.type =='Polygon')]
                important_col = ['geometry', 'building', 'building:cladding', 'building:flats', 'building:levels']
                gdf = gdf[important_col]
                max_floor = pd.DataFrame(gdf['building:levels'].value_counts()).reset_index()['index'][0]
                kind_of_build = pd.DataFrame(gdf['building'].value_counts()).reset_index()['index'][0]
                distance_road = 50
                gdf_a = ox.graph.graph_from_point(point, dist = distance_road, network_type='all')
                near_edge = ox.nearest_edges(gdf_a, x[1], x[0])
                gdf_a = ox.graph_to_gdfs(gdf_a, nodes=False)
                
                # Добавление полигона и последних координат в геодатафрейм
                context = context.append({'Имя изображения':image_name, 'geometry':point, 'Код зоны по ПЗЗ':intersecting_zones[0], 'Тип зоны':intersecting_zones[1],
                                          'Преобладающий тип застройки': kind_of_build, 'Тип ближайшей дороги': gdf_a.reset_index()['highway'].mode()[0]},
                                         ignore_index=True)
    return context

# Пример использования
text_file_path = r"C....."

# Создание геодатафрейма с полигонами
context = context_data(text_file_path)
context

In [ ]:
# Функция для нахождения ближайшего делителя числа к целевому значению
def find_closest_divisor(number, target):
    closest_divisor = 1
    closest_quotient = number
    for divisor in range(2, number + 1):
        quotient = number // divisor
        if quotient >= target and quotient < closest_quotient:
            closest_divisor = divisor
            closest_quotient = quotient
    return closest_divisor

# Функция для разделения изображения на блоки
def divide_img_blocks(img, w, h):
    horizontal = np.array_split(img, h)
    splitted_img = [np.array_split(block, w, axis=1) for block in horizontal]
    return splitted_img

# Путь к папке с изображениями
input_folder = r"C....."

# Получаем список всех файлов в папке
image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]

# Перебираем каждое изображение
for image_file in image_files:
    # Путь к текущему изображению
    image_path = os.path.join(input_folder, image_file)
    
    # Читаем изображение
    image = cv2.imread(image_path)
    
    # Получаем ширину и высоту изображения
    (width, height, _) = image.shape
    
    # Находим ближайшие делители для ширины и высоты
    w = find_closest_divisor(width, 1000)
    h = find_closest_divisor(height, 1000)

    w,h = 2,2
    
    print(f"Число блоков: ({w}, {h}) для изображения: {image_file}")
    
    # Разделяем изображение на блоки
    result = divide_img_blocks(image, w, h)
    
    # Создаем выходную директорию для сохранения блоков
    output_directory = os.path.join(input_folder, 'output_blocks')
    os.makedirs(output_directory, exist_ok=True)
    
    # Сохраняем каждый блок в отдельный файл
    for i in range(len(result)):
        for j in range(len(result[i])):
            block = result[i][j]
            output_path = os.path.join(output_directory, f"{image_file}_{i*w+j+1}.jpg")
            cv2.imwrite(output_path, block)

In [ ]:
def calculate_image_polygon_from_file(text_file_path, images_folder):
    # Создание пустого геодатафрейма
    gdf = gpd.GeoDataFrame(columns=['Image Name', 'geometry'], crs="EPSG:4326")

    # Инициализация переменных H и b
    H = None
    b = None

    # Чтение данных из файла
    with open(text_file_path, 'r') as file:
        for line in file:
            # Обработка строки с параметрами H и b
            if line.startswith('H'):
                H = float(line.strip().split()[1])
            elif line.startswith('b'):
                b = float(line.strip().split()[1])
            else:
                # Извлечение данных из строки
                image_name, coordinates = line.strip().split(' ')
                X_base, Y_base = map(float, coordinates.split(','))

                # Получение размеров изображения
                image_path = os.path.join(images_folder, image_name)
                with Image.open(image_path) as im:
                    width, height = im.size

                 # Расчет полигона и последних координат из list_A
                polygon, last_coords = calculate_image_polygon(X_base, Y_base, H, b, width, height)

                # Добавление полигона и последних координат в геодатафрейм
                gdf = gdf.append({'Image Name': image_name, 'geometry': polygon, 'Last Coordinates': last_coords}, ignore_index=True)

    # Вычисление перекрытий и добавление столбца с перекрытиями в геодатафрейм
    intersections = calculate_intersection_areas(gdf)
    gdf['Intersection Area'] = intersections

    return gdf

def calculate_image_polygon(X_base, Y_base, H, b, width, height):
    D = 2 * H * math.tan(math.radians(b/2))
    Y = (D**2 / (1 + (width/height)**2))**0.5
    X = (D**2 - Y**2)**0.5
    y = math.atan(X/Y)
    xy = []
    list_A = [y, math.pi - y, math.pi + y, 2 * math.pi - y]
    for a in list_A:
        X_coord = X_base + D * math.cos(a) / (2 * 6378136 * math.pi/180)
        Y_coord = Y_base + D * math.sin(a) / (2 * 6378136 * math.pi/180 * math.cos(X_coord))
        xy.append((Y_coord, X_coord))
    last_coords = []
    X_coord_4 = X_base + D * math.cos(a) / (2 * 6378136 * math.pi/180)
    Y_coord_4 = Y_base + D * math.sin(a) / (2 * 6378136 * math.pi/180 * math.cos(X_coord))
    last_coords.append((Y_coord_4, X_coord_4))

    # Создание полигона из точек
    polygon = Polygon(xy)
    last_coords = Point(last_coords)
    return polygon, last_coords

def calculate_intersection_areas(gdf):
    # Список для хранения полигонов перекрытий
    intersection_polygons = []

    # Подсчет перекрытий
    for index, row in gdf.iterrows():
        intersection_polygon = calculate_intersection_polygon(gdf.drop(index), row['geometry'])
        intersection_polygons.append(intersection_polygon)

    return intersection_polygons

def calculate_intersection_polygon(gdf, polygon):
    intersection_polygon = None
    for _, row in gdf.iterrows():
        if row['geometry'].intersects(polygon):
            intersection = row['geometry'].intersection(polygon)
            if not intersection.is_empty and intersection.area > 0:
                intersection_polygon = intersection
                break
    return intersection_polygon

text_file_path = r"C....."
images_folder = r"C....."

# Создание геодатафрейма с полигонами
polygons_photos = calculate_image_polygon_from_file(text_file_path, images_folder)

polygons_photos = polygons_photos.set_crs(4326)
polygons_photos

In [ ]:
polygons_photos['geometry'].explore()

In [ ]:
def extract_points_from_geometry(gdf):
    points_list = []
    image_names = []
    for idx, row in gdf.iterrows():
        geom = row['geometry']
        image_name = row['Image Name']
        if geom.type == 'Polygon':
            points = list(geom.exterior.coords)
            points_list.append(points)
            image_names.append(image_name)
        elif geom.type == 'MultiPolygon':
            for polygon in geom:
                points = list(polygon.exterior.coords)
                points_list.append(points)
                image_names.append(image_name)
    return pd.DataFrame({'Image Name': image_names, 'Points': points_list})


# Извлечение точек из геометрии и создание DataFrame
points_df = extract_points_from_geometry(polygons_photos)
points_df

In [ ]:
# Функция для вычисления точек разбиения
def calculate_split_points(points, horizontal_parts, vertical_parts):
    min_x = min(points, key=lambda p: p[0])[0]
    max_x = max(points, key=lambda p: p[0])[0]
    min_y = min(points, key=lambda p: p[1])[1]
    max_y = max(points, key=lambda p: p[1])[1]

    part_width = (max_x - min_x) / horizontal_parts
    part_height = (max_y - min_y) / vertical_parts

    parts_coordinates = []
    for i in range(vertical_parts):
        for j in range(horizontal_parts):
            part_x = min_x + j * part_width
            part_y = max_y - i * part_height
            parts_coordinates.append((part_x, part_y))
    
    return parts_coordinates

# Количество частей на горизонтали и вертикали
horizontal_parts = 2
vertical_parts = 2

# Создание пустого списка для частей
split_list = []

# Для каждой строки в points_df
for idx, row in points_df.iterrows():
    # Извлекаем имя изображения и список координат
    image_name = row['Image Name']
    points = row['Points']
    
    # Вычисляем точки разбиения
    parts_coordinates = calculate_split_points(points, horizontal_parts, vertical_parts)
    
    # Формируем и добавляем строки в список
    for i, coords in enumerate(parts_coordinates, start=1):
        # Добавляем номер части перед расширением .jpg
        image_part_name = f"{image_name.rsplit('.', 1)[0]}_{i}.{image_name.rsplit('.', 1)[1]}"
        split_list.append({'image_part_name': image_part_name, 'coords': coords})

# Создаем DataFrame из списка
split_df = pd.DataFrame(split_list)

# Выводим новый DataFrame с разбиением
split_df

In [ ]:
# Предсказание артефактов состояния газона
# Загрузка модели
model = YOLO(r"C.....") 

items = []
for root, dirs, files in os.walk(r"C.....", topdown=False):
    for name in files:
        items.append(os.path.join(root, name))

for item in items:
    results = model.predict(item, imgsz=640, conf=0.20, iou = 0.3, 
                            save = True,
                         save_txt = True
                           )

In [ ]:
# Папка с текстовыми файлами
txt_folder = r"C....."

# Читаем текстовый файл и создаем точки для каждого класса
points = []
for _, row in split_df.iterrows():
    image_name = row['image_part_name'][:-4]  # Извлекаем имя изображения без расширения
    txt_file_path = os.path.join(txt_folder, f"{image_name}.txt")
    with open(txt_file_path, "r") as file:
        lines = file.readlines()
        for line in lines:
            parts = line.split()
            class_id = int(parts[0])
            rel_x, rel_y, rel_width, rel_height = map(float, parts[1:])
            abs_x = row['coords'][0] + rel_x * rel_width
            abs_y = row['coords'][1] - rel_y * rel_height
            point = Point(abs_x, abs_y)
            points.append((row['image_part_name'], class_id, point))

# Создаем GeoDataFrame для точек
points_gdf = gpd.GeoDataFrame(points, columns=['Image Name', 'Class_ID', 'geometry'], crs='EPSG:4326')
points_gdf

In [ ]:
import pandas as pd

recommendations_df = pd.read_excel(r"C.....")

# Группировка полигонов по классу и подсчет их количества
class_counts = points_gdf.groupby(['Image Name','Class_ID']).size().reset_index(name='Count')

# Объединение с базой рекомендаций по классам
result_df = class_counts.merge(recommendations_df, on='Class_ID', how='left')

# Переименование столбца с именем фото
result_df.rename(columns={'Image Name': 'Имя фото'}, inplace=True)

# Вывод результата
result_df[result_df['Возможные условия обитания ЗН'] == 'Чрезмерная рекреационная нагрузка']

In [ ]:
import matplotlib.pyplot as plt

# Создание графика
plt.figure(figsize=(10, 6))

# Визуализация точек
plt.scatter(points_gdf['geometry'].x, points_gdf['geometry'].y, color='blue', marker='o', label='Points')

plt.title('Visualization of Points')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Отфильтровать полигоны
def filter_polygons(polygon):
    for intersection_polygon in polygons_photos['Intersection Area']:
        # Проверить пересечение с каждым полигоном из Intersection Area
        if polygon['geometry'].intersects(intersection_polygon):
                return False
    # Если пересечений меньше 50% не найдено, вернуть True
    return True

# Применить функцию к GeoDataFrame
filtered_gdf = points_gdf[points_gdf.apply(filter_polygons, axis=1)]
filtered_gdf